In [1]:
import os
# Change root directory
os.chdir("../../src/")


In [2]:
from agent_design_pattern.agent import AgentMessage, LLMChain, BaseAgent
from agent_design_pattern.orchestration import ParallelAgent
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import re

/data/agent_design_pattern/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
class CasualOllamaChain(LLMChain):
    def __init__(self, model, system_prompt, user_prompt_template = "{query}", **kargs):
        super().__init__()
        self.llm = ChatOllama(model=model, reasoning=False, **kargs) if isinstance(model, str) else model
        self.prompt = ChatPromptTemplate([
            ("system", system_prompt),
            ("human", user_prompt_template)
        ])
        self.chain = self.prompt | self.llm | StrOutputParser()

    def invoke(self, message: AgentMessage, **kwargs) -> AgentMessage:
        response = self.chain.invoke(message.to_dict(), **kwargs)
        print(response)
        # Snippet to remove thinking
        if "</think>" in response and "<think>" not in response:
            response = "<think>" + response
        response = re.sub(r'<think>.*?</think>', '', response, flags=re.DOTALL)
        message.response = response

        message.execution_result = "success"
        return message


class Agent(BaseAgent):
    def __init__(self, chain, state_change_callback = None, name = None, **kwargs):
        super().__init__(state_change_callback, name, **kwargs)
        self.chain = chain

    def execute(self, message: AgentMessage, **kwargs) -> AgentMessage:
        message = self.chain.invoke(message, **kwargs)
        message.execution_result = "success"
        message.origin = self.name
        return message

In [4]:
system_prompt = """You are a genius and creative writer.
"""

user_prompt = """{query}"""

max_iteration = 5

def state_callback(state: str):
    print(f"agent state: {state}")

chain = CasualOllamaChain(model="llama3.2:latest", system_prompt=system_prompt, user_prompt_template=user_prompt)
agent1 = Agent(chain, state_change_callback=state_callback)
agent2 = Agent(chain, state_change_callback=state_callback)
parallel_agent = ParallelAgent([agent1, agent2], state_change_callback=state_callback)

In [5]:
query = "Write a short poem with Shakespere's style about love"
message = parallel_agent.execute(AgentMessage(query=query), temperature=1.5)

print(message)
print(f"message responses len: {len(message.responses)}")

for agent_name, response in message.responses:
    print(f"{agent_name}: {response}")
    print("-" * 50)
    print()

Fairest heart, thou dost entwine
My very soul in tender twine,
A knot of passion that doth bind
And holds me fast to thy design.

Thy eyes, like sapphires shining bright,
Do sparkle with a radiant light,
That in their depths, my deepest fears
Are lost, and all my doubts do clear.

Oh, love, thou art a flame that burns
With fervent heat, and fierce concern,
For in thy touch, I find my peace,
And all the world's discord doth release.

Yet, like a tempest, thou canst rage,
And with thy fury, leave me dazed,
In stormy seas of love's dark night,
I find myself lost, without a light.

But still, I'll follow thee, fair love,
Through all the trials that life doth prove,
For in thy arms, I find my nest,
A haven from life's turbulent unrest.
O, fairest of loves, thou dost enthrall
My heart, a captive to thy sweet call
Like summer's warmth, thy touch doth ignite
A flame that burns, and all my senses excite

Thy eyes, like sapphires shining bright and blue
Do sparkle with a love most true and new
T